## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,2021-11-02 03:54:49,42.0970,-79.2353,42.03,84,90,1.99,overcast clouds
1,1,Bethel,US,2021-11-02 03:54:49,41.3712,-73.4140,48.49,79,99,3.40,overcast clouds
2,2,Torbay,CA,2021-11-02 03:54:50,47.6666,-52.7314,52.75,96,90,8.05,mist
3,3,Albany,US,2021-11-02 03:53:20,42.6001,-73.9662,42.53,77,100,5.99,overcast clouds
4,4,Cidreira,BR,2021-11-02 03:54:52,-30.1811,-50.2056,68.31,93,80,17.69,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# 4a. Determine if there are any empty rows.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>=min_temp) & (city_data_df["Max Temp"]<=max_temp)]
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                9
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [5]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bethel,US,48.49,overcast clouds,41.3712,-73.4140,
2,Torbay,CA,52.75,mist,47.6666,-52.7314,
4,Cidreira,BR,68.31,broken clouds,-30.1811,-50.2056,
8,Puerto Ayora,EC,68.45,broken clouds,-0.7393,-90.3518,
9,Saint George,US,65.53,clear sky,37.1041,-113.5841,
11,Kaitangata,NZ,51.69,broken clouds,-46.2817,169.8464,
12,Ushuaia,AR,49.66,clear sky,-54.8000,-68.3000,
13,Menongue,AO,61.84,scattered clouds,-14.6585,17.6910,
14,Chapada Dos Guimaraes,BR,71.26,broken clouds,-15.4606,-55.7497,
15,Saint-Francois,FR,47.48,clear sky,46.4154,3.9054,


In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bethel,US,48.49,overcast clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
2,Torbay,CA,52.75,mist,47.6666,-52.7314,Gallows Cove
4,Cidreira,BR,68.31,broken clouds,-30.1811,-50.2056,Hotel Castelo
8,Puerto Ayora,EC,68.45,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
9,Saint George,US,65.53,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
11,Kaitangata,NZ,51.69,broken clouds,-46.2817,169.8464,Kaitangata Motor Camp
12,Ushuaia,AR,49.66,clear sky,-54.8000,-68.3000,Albatros Hotel
13,Menongue,AO,61.84,scattered clouds,-14.6585,17.6910,Ritz Lauca
14,Chapada Dos Guimaraes,BR,71.26,broken clouds,-15.4606,-55.7497,Pousada Morro Dos Ventos
15,Saint-Francois,FR,47.48,clear sky,46.4154,3.9054,Manoir Du Bonheur


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))